In [1]:
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies')

import alchemy_tools as at
from geometry_euston import abc_to_hmatrix, distance_pbc
from parse_cube_files import CUBE

import numpy as np

Calculation of the distance between a nucleus and a group of vectors represented on a grid using the minimal image convention

In [2]:
def distance_MIC(pos_nuc, meshgrid, h_matrix):
    """
    calculates the distance between the position of the nucleus and the nearest image of a gridpoint
    
    pos_nuc: position of nucleus
    meshgrid: meshgrid containing x,y,z components of every gridpoint
    h_matrix: needed for calculation of MIC distance
    """
    
    distance = np.zeros((meshgrid[0].shape))
    
    for idx0 in range(meshgrid[0].shape[0]):
        for idx1 in range(meshgrid[0].shape[1]):
            for idx2 in range(meshgrid[0].shape[2]):
                distance[idx0][idx1][idx2] = distance_pbc(pos_nuc, np.array([meshgrid[0][idx0][idx1][idx2], meshgrid[1][idx0][idx1][idx2], meshgrid[2][idx0][idx1][idx2]]), h_matrix)
    
    return(distance)

In [3]:
def distance_MIC2(pos_nuc, meshgrid, h_matrix):
    """
    calculates the distance between the position of the nucleus and the nearest image of a gridpoint
    works so far only for cubic symmetry
    
    pos_nuc: position of nucleus
    meshgrid: meshgrid containing x,y,z components of every gridpoint
    h_matrix: needed for calculation of MIC distance
    :return: distance between pos_nuc and every gridpoint
    :rtype: numpy array of shape meshgrid.shape
    """
    
    hinv = np.linalg.inv(h_matrix)
    a_t = np.dot(hinv, pos_nuc)
    
    # calculate product of h_matrix and grid componentwise
    b_t_x = hinv[0][0]*meshgrid[0]
    b_t_y = hinv[1][1]*meshgrid[1]
    b_t_z = +hinv[2][2]*meshgrid[2]
    
    t_12_x = b_t_x - a_t[0]
    t_12_y = b_t_y - a_t[1]
    t_12_z = b_t_z - a_t[2]
    
    t_12_x -= np.round(t_12_x)
    t_12_y -= np.round(t_12_y)
    t_12_z -= np.round(t_12_z)
    
    x = np.power(h_matrix[0][0]*t_12_x, 2)
    y = np.power(h_matrix[1][1]*t_12_y, 2)
    z = np.power(h_matrix[2][2]*t_12_z, 2)
    
    return(np.sqrt(x+y+z))

In [4]:
cube = CUBE('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/ve_04.cube')

In [21]:
# abc_to_hmatrix(20, 20, 20, 90, 90, 90)
h_matrix = np.array([cube.X*cube.NX, cube.Y*cube.NY, cube.Z*cube.NZ])
h_matrix


array([[37.794575,  0.      ,  0.      ],
       [ 0.      , 37.794575,  0.      ],
       [ 0.      ,  0.      , 37.794575]])

In [6]:
hinv = np.linalg.inv(h_matrix)
a = cube.atoms[0][1:4]
a_t = np.dot(hinv, a)

In [7]:
hinv[0][0]

0.026458823786217993

In [8]:
meshgrid = cube.get_grid()

In [21]:
%timeit distances_mic = distance_MIC(cube.atoms[0][1:4], meshgrid, h_matrix)

1min 44s ± 1.75 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
distances_mic = distance_MIC(cube.atoms[0][1:4], meshgrid, h_matrix)

In [10]:
%timeit distance2 = distance_MIC2(cube.atoms[0][1:4], meshgrid, h_matrix)

250 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
distance_mic2 = distance_MIC2(cube.atoms[0][1:4], meshgrid, h_matrix)

In [13]:
np.allclose(distances_mic,distance_mic2)

True

In [54]:
np.amax(distance_mic2)

32.72340188943374

In [48]:
meshgrid_xyz = np.vstack([_.flatten() for _ in meshgrid]).T
dist_gpt_nuc = np.linalg.norm(meshgrid_xyz - cube.atoms[0][1:4], axis=1)

In [53]:
np.amax(dist_gpt_nuc)

35.7188390617939

Calculation of the distance with MIC

In [2]:
import alchemy_tools as at

In [3]:
base = '/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/'
filenames = [('ve_08.cube', 8/38), ('ve_15.cube', 15/38), ('ve_23.cube', 23/38), ('ve_30.cube', 30/38),('ve_38.cube', 38/38)]
for idx,i in enumerate(filenames):
    filenames[idx] = (base+i[0], i[1])
cube_dirs = filenames
cube_dirs.insert(0, ('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/ueg/ve_00.cube', 0))

In [4]:
filenames

[('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/ueg/ve_00.cube',
  0),
 ('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/ve_08.cube',
  0.21052631578947367),
 ('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/ve_15.cube',
  0.39473684210526316),
 ('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/ve_23.cube',
  0.6052631578947368),
 ('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/ve_30.cube',
  0.7894736842105263),
 ('/home/misa/APDFT/prototyping/atomic_energies/results/slice_ve38/dsgdb9nsd_001212/cube-files/ve_38.cube',
  1.0)]

In [5]:
nucs, energies, alchpots = at.atomic_energy_decomposition(cube_dirs)

In [6]:
energies

array([-37.2373768 , -43.97885181, -57.86113179, -62.84254247,
       -41.48332069, -38.48330555, -41.5503646 ,  -5.27404281,
        -5.39576267,  -5.39437558,  -5.91036065,  -5.91095847])

In [7]:
energies.sum()

-351.3223938950071

In [8]:
alchpots

array([-6.20622947, -7.32980863, -7.23264147, -7.85531781, -6.91388678,
       -6.41388426, -5.93576637, -5.27404281, -5.39576267, -5.39437558,
       -5.91036065, -5.91095847])

In [9]:
nucs

array([[ 6.      , 19.979529, 22.784627, 18.908058],
       [ 6.      , 20.35982 , 19.964565, 18.890009],
       [ 8.      , 22.344159, 18.857926, 18.863323],
       [ 8.      , 18.092893, 18.74945 , 18.905025],
       [ 6.      , 18.285827, 16.053484, 18.887816],
       [ 6.      , 15.726597, 15.015175, 18.906865],
       [ 7.      , 13.735016, 14.125508, 18.920052],
       [ 1.      , 21.813395, 23.718436, 18.8968  ],
       [ 1.      , 18.878631, 23.358359, 17.25609 ],
       [ 1.      , 18.90752 , 23.341078, 20.584704],
       [ 1.      , 19.30598 , 15.408752, 17.207074],
       [ 1.      , 19.337767, 15.389776, 20.541321]])